<a href="https://colab.research.google.com/github/WeustiS/ProjectX/blob/main/Data_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests as r
from io import BytesIO, StringIO
import zipfile
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from google.colab import drive
import time

In [3]:
base_url = r'http://oasis.caiso.com/oasisapi/SingleZip?'
# ?queryname=PRC_AS&startdatetime=20191018T07:00-0000&enddatetime=20191019T07:00-0000
# &market_run_id=DAM&version=1&as_type=ALL&as_region=ALL

def get_data(params):
    # building URL
    url = base_url
    for key, value in params.items():
        url += f"{key}={value}&"
    data = r.get(url[:-1]).content
    print("URL:", url)
    try:
        filecontent = BytesIO(data)
    except TypeError:
        filecontent = StringIO(data)

    try:
            # have zipfile
        z = zipfile.ZipFile(filecontent)
    except zipfile.BadZipfile:
        return None

        # have unzipped content
    unzipped = [z.read(thisfile) for thisfile in z.namelist()]
    z.close()

        # return
    return unzipped

In [4]:
# function to convert XML to 
def parse_XML(root):
  df = pd.DataFrame(columns=['Variable', 'Node', 'Date','Interval Number','Value'])
  j = 0
  #
  for i in root[1][0][4][1:]:
    #print(i[2].text, float(i[6].text))\
    new_row = {'Variable':i[0].text, 'Node':i[1].text, 'Date':i[2].text, 'Interval Number':float(i[3].text),'Value':float(i[6].text)}
    #append row to the dataframe
    df = df.append(new_row, ignore_index=True)
  return df

In [5]:
#trial cell
#payload = {
#            'queryname': 'PRC_INTVL_LMP', 
#            'startdatetime': '20200102T08:00-0000',
#            'enddatetime':'20200103T08:00-0000',
#            'version': '1',
#            'market_run_id': 'RTM',
#            'node': 'PERRY_6_N001'
#          }
##xml = get_data(payload)
#xml_to_string = str(xml[0],'utf-8')
#display(xml_to_string)
#root = ET.fromstring(xml_to_string)
#data = parse_XML(root)
##DF_list.append(data)


URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20200102T08:00-0000&enddatetime=20200103T08:00-0000&version=1&market_run_id=RTM&node=PERRY_6_N001&


'<?xml version="1.0" encoding="UTF-8"?><OASISReport xmlns="http://www.caiso.com/soa/OASISReport_v1.xsd">\n<MessageHeader>\n<TimeDate>2020-10-30T14:58:36-00:00</TimeDate>\n<Source>OASIS</Source>\n<Version>v20131201</Version>\n</MessageHeader>\n<MessagePayload>\n<RTO>\n<name>CAISO</name>\n<REPORT_ITEM>\n<REPORT_HEADER>\n<SYSTEM>OASIS</SYSTEM>\n<TZ>PPT</TZ>\n<REPORT>PRC_INTVL_LMP</REPORT>\n<MKT_TYPE>RTM</MKT_TYPE>\n<UOM>US$/MWh</UOM>\n<INTERVAL>BEGINNING</INTERVAL>\n<SEC_PER_INTERVAL>300</SEC_PER_INTERVAL>\n</REPORT_HEADER>\n<REPORT_DATA>\n<DATA_ITEM>LMP_CONG_PRC</DATA_ITEM>\n<RESOURCE_NAME>PERRY_6_N001</RESOURCE_NAME>\n<OPR_DATE>2020-01-02</OPR_DATE>\n<INTERVAL_NUM>1</INTERVAL_NUM>\n<INTERVAL_START_GMT>2020-01-02T08:00:00-00:00</INTERVAL_START_GMT>\n<INTERVAL_END_GMT>2020-01-02T08:05:00-00:00</INTERVAL_END_GMT>\n<VALUE>0</VALUE>\n</REPORT_DATA>\n<REPORT_DATA>\n<DATA_ITEM>LMP_CONG_PRC</DATA_ITEM>\n<RESOURCE_NAME>PERRY_6_N001</RESOURCE_NAME>\n<OPR_DATE>2020-01-02</OPR_DATE>\n<INTERVAL_NUM>

In [6]:
nodes_list_1 = ['WOHLER_6_N001', 'SOLEDAD_6_N001', 'ZAMORA_1_N001', 'SILVERDO_1_N002', 'ZANKER_1_N003']
nodes_list_2 = ['WINTERS_6_N001', 'REDBUD_1_N004']
nodes_list_3 = ['PERRY_6_N001', 'SANDBAR_7_B1', 'TULUCAY_6_N001']

In [7]:
DF_list= list()
for node in nodes_list_1:
  time.sleep(2)
  for month in range(1, 13):
    time.sleep(2)
    for i in range(1,31):
      if (month == 2 and i > 27):
        continue
      if ((month == 4 or month == 6 or month == 9 or month == 11) and i > 29):
        continue
      time.sleep(5)
      if (month < 10):
        if (i < 10):
          # changing the start and end date for scraping purposes
          start_date = '20190' + str(month)+'0' + str(i)+'T08:00-0000'
          end_date = '20190'+str(month) + '0'+str(i+1)+'T08:00-0000'
          # dynamically changing payload
          payload = {
            'queryname': 'PRC_INTVL_LMP', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': 'RTM',
            'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          # converting xml to dataframe
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML(root)
          DF_list.append(data)
        else:
          start_date = '20190' + str(month)+ str(i)+'T08:00-0000'
          end_date = '20190'+str(month) + str(i+1)+'T08:00-0000'
          payload = {
            'queryname': 'PRC_INTVL_LMP', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': 'RTM',
            'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML(root)
          DF_list.append(data)
      else:
        if (i < 10):
          # changing the start and end date for scraping purposes
          start_date = '2019' + str(month)+ '0'+ str(i)+'T08:00-0000'
          end_date = '2019'+str(month)+'0'+str(i+1)+'T08:00-0000'
          # dynamically changing payload
          payload = {
            'queryname': 'PRC_INTVL_LMP', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
          'market_run_id': 'RTM',
          'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          # converting xml to dataframe
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML(root)
          DF_list.append(data)
        else:
          start_date = '2019' + str(month)+ str(i)+'T08:00-0000'
          end_date = '2019' + str(month)+str(i+1)+'T08:00-0000'
          payload = {
            'queryname': 'PRC_INTVL_LMP', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': 'RTM',
            'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML(root)
          DF_list.append(data)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20200101T08:00-0000&enddatetime=20200102T08:00-0000&version=1&market_run_id=RTM&node=PERRY_6_N001&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20200102T08:00-0000&enddatetime=20200103T08:00-0000&version=1&market_run_id=RTM&node=PERRY_6_N001&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20200103T08:00-0000&enddatetime=20200104T08:00-0000&version=1&market_run_id=RTM&node=PERRY_6_N001&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20200104T08:00-0000&enddatetime=20200105T08:00-0000&version=1&market_run_id=RTM&node=PERRY_6_N001&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20200105T08:00-0000&enddatetime=20200106T08:00-0000&version=1&market_run_id=RTM&node=PERRY_6_N001&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdate

In [8]:
DF_list
df = pd.concat(DF_list)
df

,Variable,Node,Date,Interval Number,Value
0,LMP_PRC,PERRY_6_N001,2020-01-01,1.0,25.44314
1,LMP_PRC,PERRY_6_N001,2020-01-01,2.0,26.76972
2,LMP_PRC,PERRY_6_N001,2020-01-01,3.0,25.79822
3,LMP_PRC,PERRY_6_N001,2020-01-01,4.0,25.50110
4,LMP_PRC,PERRY_6_N001,2020-01-01,5.0,25.86202
...,...,...,...,...,...
271,LMP_PRC,TULUCAY_6_N001,2020-10-28,284.0,32.61927
272,LMP_PRC,TULUCAY_6_N001,2020-10-28,285.0,32.54263
273,LMP_PRC,TULUCAY_6_N001,2020-10-28,286.0,31.89034
274,LMP_PRC,TULUCAY_6_N001,2020-10-28,287.0,31.72615


In [9]:
# code to save data in google drive directly
#data = parse_XML(root)
drive.mount('drive', force_remount=True)
df.to_csv('2020_3_nodes.csv', index = False)
!cp 2020_3_nodes.csv "drive/My Drive/ProjectX Data"

Mounted at drive
